In [1]:
import pandas as pd
import numpy as np
from pyteomics import pepxml, mzid
import re

In [30]:
import itertools

In [20]:
import xml.etree.cElementTree as ET
from xml.etree.cElementTree import iterparse

In [2]:
import pkg_resources

In [3]:
import click

In [4]:
unimodfile = pkg_resources.resource_filename('easypqp', 'data/unimod.xml')

In [5]:
class unimod:
	def __init__(self, unimod_file, max_delta):
		self.unimod_file = unimod_file
		self.max_delta = max_delta
		self.ptms = self.parse_unimod()

	def parse_unimod(self):
		namespaces = {'umod': "http://www.unimod.org/xmlns/schema/unimod_2"}
		ET.register_namespace('', "http://www.unimod.org/xmlns/schema/unimod_2")
		tree = ET.parse(self.unimod_file)
		root = tree.getroot()

		ptms = {}
		sites = ['A','R','N','D','C','E','Q','G','H','O','I','L','K','M','F','P','U','S','T','W','Y','V','N-term','C-term']
		positions = ['Anywhere','Any N-term','Any C-term','Protein N-term','Protein C-term']

		for site in sites:
			ptms[site] = {}

		for position in positions:
			for site in sites:
				ptms[site][position] = {}

		for modifications in root.findall('.//umod:modifications', namespaces):
			for modification in modifications.findall('.//umod:mod', namespaces):
				for specificity in modification.findall('.//umod:specificity', namespaces):
					ptms[specificity.attrib['site']][specificity.attrib['position']][int(modification.attrib['record_id'])] = float(modification.findall('.//umod:delta', namespaces)[0].attrib['mono_mass'])

		return ptms

	def get_id(self, site, position, delta_mass):
		candidates = {}
		min_id = -1
		ptms_site = self.ptms[site]
		search_multiple_positions = isinstance(position, (list, tuple))

		kvs = itertools.chain.from_iterable((((k, p), v) for k,v in ptms_site[p].items()) for p in position) \
			if search_multiple_positions else \
			ptms_site[position].items()
		for key, value in kvs:
			delta_mod = abs(value - float(delta_mass))
			if delta_mod < self.max_delta:
				if key in candidates.keys():
					if delta_mod < candidates[key]:
						candidates[key] = delta_mod
				else:
					candidates[key] = delta_mod

		if len(candidates) > 0:
			min_id = min(candidates, key=candidates.get)

		return(min_id)

	def get_oms_id(self, sequence, massdiff, nterm_modification, cterm_modification):
		record_ids = {}
		for site, aa in enumerate(sequence):
			if aa != "_":
				record_id_site = self.get_id(aa, 'Anywhere', massdiff)
				if record_id_site != -1:
					record_ids[site+1] = record_id_site

		record_id_nterm = -1
		if nterm_modification == "":
			record_id_nterm = self.get_id("N-term", 'Any N-term', massdiff)
			if record_id_nterm == -1:
				record_id_nterm = self.get_id("N-term", 'Protein N-term', massdiff)

		record_id_cterm = -1
		if cterm_modification == "":
			record_id_cterm = self.get_id("C-term", 'Any C-term', massdiff)
			if record_id_cterm == -1:
				record_id_cterm = self.get_id("C-term", 'Protein C-term', massdiff)

		# prefer residual over N-term over C-term modifications
		aamod = {}
		if len(record_ids) > 0:
			aasite = median_low(list(record_ids.keys()))
			aamod[aasite] = massdiff
		elif record_id_nterm != -1:
			nterm_modification = massdiff
		elif record_id_cterm != -1:
			cterm_modification = massdiff

		return aamod, nterm_modification, cterm_modification

In [6]:
enable_massdiff = False
exclude_range = [-1.5, 3.5]
enable_unannotated = False
def match_unimod(unimod, psms):
    def match_modifications(um, peptide):
      
        monomeric_masses = {"A": 71.03711, "R": 156.10111, "N": 114.04293, "D": 115.02694, "C": 103.00919, "E": 129.04259, "Q": 128.05858, "G": 57.02146, "H": 137.05891, "I": 113.08406, "L": 113.08406, "K": 128.09496, "M": 131.04049, "F": 147.06841, "P": 97.05276, "S": 87.03203, "T": 101.04768, "W": 186.07931, "Y": 163.06333, "V": 99.06841}
        modified_peptide = peptide['peptide_sequence']

        # parse terminal modifications
        nterm_modification = ""
        if peptide['nterm_modification'] is not "":
            nterm_modification = peptide['nterm_modification'] - 1.0078
        cterm_modification = ""
        if peptide['cterm_modification'] is not "":
            cterm_modification = peptide['cterm_modification']

        # parse closed modifications
        modifications = {}
        if "M|" in peptide['modifications']:
            for modification in peptide['modifications'].split('|')[1:]:
                site, mass = modification.split('$')
                delta_mass = float(mass) - monomeric_masses[peptide['peptide_sequence'][int(site)-1]]
                modifications[int(site)] = delta_mass

        massdiff = float(peptide['massdiff'])
        if enable_massdiff and (massdiff < exclude_range[0] or massdiff > exclude_range[1]):
            # parse open modifications
            oms_sequence = peptide['peptide_sequence']
            for site in modifications.keys():
                oms_sequence = oms_sequence[:site-1] + "_" + oms_sequence[site:]

            oms_modifications, nterm_modification, cterm_modification = um.get_oms_id(oms_sequence, peptide['massdiff'], nterm_modification, cterm_modification)
            modifications = {**modifications, **oms_modifications}

        peptide_sequence = peptide['peptide_sequence']
        peptide_length = len(peptide_sequence)
        for i, site in enumerate(sorted(modifications, reverse=True)):
            positions = ('Anywhere', 'Any N-term', 'Protein N-term') if site == 1 else \
                ('Anywhere', 'Any C-term', 'Protein C-term') if site == peptide_length else \
                    'Anywhere'
            record_id0 = um.get_id(peptide_sequence[site - 1], positions, modifications[site])
            if isinstance(record_id0, tuple):
                record_id, position = record_id0
            else:
                record_id = record_id0
            is_N_term = isinstance(record_id0, tuple) and position in ('Any N-term', 'Protein N-term')
            

            if record_id == -1:
                
                if enable_unannotated:
                    modified_peptide = "[" + str(massdiff) + "]" + modified_peptide \
                    if is_N_term else \
                    modified_peptide[:site] + "[" + str(massdiff) + "]" + modified_peptide[site:]
                else:
                    raise click.ClickException("Error: Could not annotate site %s (%s) from peptide %s with delta mass %s." % (site, peptide['peptide_sequence'][site-1], peptide['peptide_sequence'], modifications[site]))
            else:
                modified_peptide = "(UniMod:" + str(record_id) + ")" + modified_peptide \
                    if is_N_term else \
                    modified_peptide[:site] + "(UniMod:" + str(record_id) + ")" + modified_peptide[site:]

        if nterm_modification is not "":
            record_id_nterm = um.get_id("N-term", 'Any N-term', nterm_modification)
            if record_id_nterm == -1:
                record_id_nterm = um.get_id("N-term", 'Protein N-term', nterm_modification)

            if record_id_nterm == -1:
                if enable_unannotated:
                    modified_peptide = ".[" + str(massdiff) + "]" + modified_peptide
                else:
                    raise click.ClickException("Error: Could not annotate N-terminus from peptide %s with delta mass %s." % (peptide['peptide_sequence'], nterm_modification))
            else:
                modified_peptide = ".(UniMod:" + str(record_id_nterm) + ")" + modified_peptide

        if cterm_modification is not "":
            record_id_cterm = um.get_id("C-term", 'Any C-term', cterm_modification)
            if record_id_cterm == -1:
                record_id_cterm = um.get_id("C-term", 'Protein C-term', cterm_modification)

            if record_id_cterm == -1:
                if enable_unannotated:
                    modified_peptide = modified_peptide + ".[" + str(massdiff) + "]"
                else:
                    raise click.ClickException("Error: Could not annotate C-terminus from peptide %s with delta mass %s." % (peptide['peptide_sequence'], cterm_modification))
            else:
                modified_peptide = modified_peptide + ".(UniMod:" + str(record_id_cterm) + ")"


        return modified_peptide


    if psms.shape[0] > 0:
        psms['modified_peptide'] = psms[['peptide_sequence','modifications','nterm_modification','cterm_modification','massdiff']].apply(lambda x: match_modifications(unimod, x), axis=1)
        
    return psms

In [8]:
class pepxmlDfExtracter:
    def __init__(self, df):
        self.df = df
        self.spectrumDict = self._getSpectrumDict()
        
    def _checkLength(self, lists):
        it = iter(lists)
        the_len = len(next(it))
        if not all(len(l) == the_len for l in it):
             raise ValueError('not all lists have same length!')
        
        
    def _getSpectrumDict(self):
        SpectrumQuery = self._getSpectrum_query_dict()
        SearchHit = self._getSearch_hit_dict()
        ModiefiedProteins = self._getModifiedDicts()
        SearchScores = self._getSearchScores()
        PeptideProphet = self._getPeptideProphetResult()
        ScoreSummary = self._getScoreSummary()
        
        self._checkLength([SpectrumQuery, SearchHit, ModiefiedProteins, SearchScores, PeptideProphet, ScoreSummary])
        
        spectrumDict = dict()
        for i in range(len(SpectrumQuery)):
            spectrumDict["spectrum{}".format(i)] = {
                "spectrumQuery": SpectrumQuery[i],
                "searchHit": SearchHit[i],
                "modiefiedProteins": ModiefiedProteins[i],
                "searchScores": SearchScores[i],
                "peptideProphet": PeptideProphet[i],
                "scoreSummary": ScoreSummary[i],
            }
        
        return spectrumDict
        
        
    def _getSpectrum_query_dict(self):
        spectrum_query_var = ["start_scan", "assumed_charge", "native_id", 
                              "spectrum", "end_scan", "index", 
                              "precursor_neutral_mass", "retention_time_sec"]
        
        list_of_dicts = list()
        for values in self.df[spectrum_query_var].values:
            spectrum_query_dict = dict(zip(spectrum_query_var, values))

            native_id = spectrum_query_dict["native_id"]
            scan = re.search(r'scan=(.*?)$', native_id).group(1)

            spectrum_query_dict["scan"] = scan

            list_of_dicts.append(spectrum_query_dict)
        return list_of_dicts
    
    def _getSearch_hit_dict(self):
    
        search_hit_var = ["peptide", "massdiff", "calc_neutral_pep_mass", 
                          "num_missed_cleavages", "num_tot_proteins", "tot_num_ions",
                          "hit_rank", "num_matched_ions", "is_rejected"]

        protein_info_var = ["protein", "protein_descr", "num_tol_term", "peptide_prev_aa", "peptide_next_aa"]


        searchInfoList = list()
        for searchInfo, ProtInfo in zip(self.df[search_hit_var].values, self.df[protein_info_var].values):
            InfoDict = dict()
            InfoDict["alternativeProteins"] = None

            alternativeProt = list()

            searchInfoDict = dict(zip(search_hit_var, searchInfo))
            for i, t in enumerate(list(zip(ProtInfo[0], ProtInfo[1] , ProtInfo[2], ProtInfo[3], ProtInfo[4]))):
                protInfoDict = dict(zip(protein_info_var, t))
                if i == 0:
                    InfoDict["mainProtein"] = dict(searchInfoDict, **protInfoDict)
                else:
                    alternativeProt.append(protInfoDict)

            if i > 0:
                InfoDict["alternativeProteins"] = alternativeProt


            searchInfoList.append(InfoDict)
    
        return searchInfoList
    
    
    def _getModifiedDicts(self):
        modified_peptide_list = list()
        for modified_peptide, values in zip(self.df.modified_peptide.values, self.df.modifications.values):
            modified_peptide_dict = dict()
            mod_aminoacid_mass_list = list()
            for v in values:
                mass, pos = v.split('@')
                mod_aminoacid_mass_list.append({"position": pos, "mass": mass})

            if len(mod_aminoacid_mass_list)>0:
                modified_peptide_dict[modified_peptide] = mod_aminoacid_mass_list
            else:
                modified_peptide_dict[modified_peptide] = None
    
            modified_peptide_list.append(modified_peptide_dict)
    
        return modified_peptide_list
    
    def _getSearchScores(self):
        search_score_var = ["hyperscore", "nextscore", "expect"]
        searchScoreList = list()
        for values in self.df[search_score_var].values:
            searchScoreList.append(dict(zip(search_score_var, values)))
        return searchScoreList
    
    def _getPeptideProphetResult(self):
        search_score_var = ["peptideprophet_probability", "peptideprophet_ntt_prob"]
        names = ["probability", "all_ntt_prob"]
    
        searchScoreList = list()
        for values in self.df[search_score_var].values:
            searchScoreList.append(dict(zip(names, values)))
        return searchScoreList
    
    def _getScoreSummary(self):
        search_score_var = ["fval", "ntt", "nmc", "massd", "isomassd"]

        searchScoreList = list()
        for values in self.df[search_score_var].values:
            searchScoreList.append(dict(zip(search_score_var, values)))
        return searchScoreList


In [24]:
class buildPSM(pepxmlDfExtracter):
    def __init__(self, df):
        super().__init__(df)
        
        
    def _getProtGeneIds(self, data):
        decoy_prefix = "rev_"

        unprocessed_proteins = [data["mainProtein"]["protein"]]
        alternativeProteins = data["alternativeProteins"]
        if alternativeProteins:
            for aProteins in alternativeProteins:
                unprocessed_proteins.append(aProteins['protein'])

        has_targets = False
        has_decoys = False
        for prot in unprocessed_proteins:
            if decoy_prefix in prot:
                has_decoys = True
            else:
                has_targets = True

        processed_proteins = []
        for prot in unprocessed_proteins:
            if has_targets and has_decoys:
                if decoy_prefix not in prot:
                    processed_proteins.append(prot)
            else:
                processed_proteins.append(prot)
        num_tot_proteins = len(processed_proteins)


        is_decoy = False
        if has_decoys and not has_targets:
            is_decoy = True

        proteins = {}
        for prot in processed_proteins:
            # Remove UniProt prefixes if necessary
            if decoy_prefix + "sp|" in prot:
                proteins[decoy_prefix + prot.split("|")[1]] = ""
            elif "sp|" in prot:
                proteins[prot.split("|")[1]] = prot.split("|")[2].split(" ")[0].split("_")[0]
            else:
                proteins[prot] = prot

        protein = ""
        gene = ""

        for key in sorted(proteins):
            if protein == "":
                protein = key
            else:
                protein = protein + ";" + key

            if gene == "":
                gene = proteins[key]
            else:
                gene = gene + ";" + proteins[key]
        return protein, gene, num_tot_proteins, is_decoy
    
    #values["modiefiedProteins"]
    def _getModification(self, data, pep_len):
        nterm_modification = ""
        cterm_modification = ""

        modifications = "M"
        if list(data.values())[0]:
            for v in list(data.values())[0]:
                if int(v['position']) == 0:
                    nterm_modification = float(v["mass"])
                elif int(v['position']) == pep_len:
                    cterm_modification = float(v["mass"])
                else:
                    modifications = modifications + "|" + v['position'] + "$" + v["mass"]
        
        return modifications, nterm_modification, cterm_modification
        

        
    def build_psm(self):
        dictList = list()
        
        for key, values in self.spectrumDict.items():
            base_name = values["spectrumQuery"]["spectrum"].split(".")[0]
            scan_id = int(values["spectrumQuery"]["start_scan"])


            hit_rank = int(values["searchHit"]["mainProtein"]["hit_rank"])
            massdiff = float(values["searchHit"]["mainProtein"]["massdiff"])

            precursor_charge = int(values["spectrumQuery"]["assumed_charge"])
            retention_time = values["spectrumQuery"]['retention_time_sec']
            ion_mobility = np.nan
            peptide_sequence = values["searchHit"]["mainProtein"]["peptide"]
            
            
            modifications, nterm_modification, cterm_modification = self._getModification(values["modiefiedProteins"], len(peptide_sequence))

            protein, gene, num_tot_proteins, is_decoy = self._getProtGeneIds(values["searchHit"])

            var_hyperscore = values["searchScores"]["hyperscore"]
            var_nextscore = values["searchScores"]["nextscore"]
            var_expect = values["searchScores"]["expect"]

            pep = 1.0 - float(values["peptideProphet"]['probability'])

            d = {'run_id': base_name, 
                 'scan_id': int(scan_id), 
                 'hit_rank': int(hit_rank),
                 'massdiff': float(massdiff), 

                 'precursor_charge': int(precursor_charge), 
                 'retention_time': float(retention_time), 
                 'ion_mobility': float(ion_mobility), 
                 'peptide_sequence': peptide_sequence, 
                 'modifications': modifications, 
                 'nterm_modification': nterm_modification, 
                 'cterm_modification': cterm_modification, 
                 'protein_id': protein,
                 'gene_id': gene, 
                 'num_tot_proteins': num_tot_proteins, 
                 'decoy': is_decoy,
                 'var_hyperscore':var_hyperscore, 
                 'var_nextscore': var_nextscore,
                 'var_expect': var_expect,
                 'pep': pep 
                }

            dictList.append(d)
        return pd.DataFrame(dictList)


In [25]:
iProphet = pepxml.DataFrame("/home/ekvall/Desktop/output/interact-Sample1.pep.xml", )

In [26]:
bpsm = buildPSM(iProphet)

In [27]:
df = bpsm.build_psm()

In [28]:
max_delta_unimod = 0.02
um = unimod(unimodfile, max_delta_unimod)

In [31]:
psms = match_unimod(um, df)

In [33]:
path = "/home/ekvall/easypqp/easypqp/"
p_proph = pd.read_csv(path + "iProphet_df", index_col=0)


In [36]:
p_proph = p_proph.sort_values("scan_id")
psms = psms.sort_values("scan_id")

In [39]:
for c in p_proph.columns:
    if psms[c].dtype == np.object:
        if (~(psms[c] == p_proph[c])).sum() > 0:
            print("Baaaad",c)
        else:
            print("Goood",c)
            
    else:
        if np.allclose(psms[c], p_proph[c]):
            print("Goood",c)
        else:
            print("Baaaad",c)
        

Goood run_id
Goood scan_id
Goood hit_rank
Goood massdiff
Goood precursor_charge
Goood retention_time
Baaaad ion_mobility
Goood peptide_sequence
Goood modifications
Baaaad nterm_modification
Baaaad cterm_modification
Goood protein_id
Baaaad gene_id
Goood num_tot_proteins
Goood decoy
Goood var_hyperscore
Goood var_nextscore
Goood var_expect
Goood pep
Goood modified_peptide


In [ ]:
def getModifiedDicts(df):
    modified_peptide_list = list()
    for values in df.modified_peptide.values:
        modified_peptide_dict = dict()
        mod_aminoacid_mass_list = list()
        for m in re.finditer('\[(.*?)\]', values):
            mod_aminoacid_mass_list.append({"position": m.start(), "mass": m.group(1)})

        if len(mod_aminoacid_mass_list)>0:
            modified_peptide_dict[values] = mod_aminoacid_mass_list
        else:
            modified_peptide_dict[values] = None
    
        modified_peptide_list.append(modified_peptide_dict)
    
    return modified_peptide_list

In [ ]:
def getSearch_hit_dict(df, spectrum_query_var):
    list_of_dicts = list()
    for values in df[spectrum_query_var].values:
        search_hit_dict = dict(zip(spectrum_query_var, values))
        
        list_of_dicts.append(search_hit_dict)
    return list_of_dicts

In [ ]:
class pepxmlBuilder(pepxmlDfExtracter):
    def __init__(self, df):
        super().__init__(df)
        
        self.spectrumQueryEndTag = "</spectrum_query>"
        
        self.searchResultStartTag = "<search_result>"
        self.searchResultEndTag = "</search_result>"
        
        self.searchHitEndTag = "</search_hit>"
        
        self.modification_infoEndtag = "</modification_info>"
        
        self.analysisResultEndTag = "</peptideprophet_result></analysis_result>"
        
        self.getScoreSummaryEndTag = "</search_score_summary>"
        
        
        
    def getSpectrumQueryStartTag(self, data):
        startTag = """<spectrum_query start_scan="{}" assumed_charge="{}" native_id="{}" spectrum="{}" end_scan="{}" index="{}" precursor_neutral_mass="{}" retention_time_sec="{}">""".format(data["start_scan"], 
                                           data["assumed_charge"],
                                           data["native_id"],
                                           data["spectrum"],
                                           data["end_scan"],
                                           data["index"],
                                           data["precursor_neutral_mass"],
                                           data["retention_time_sec"],
                                           data["scan"])
        return startTag
    
    def getSearchHitStartTag(self, data):
        startTag = """<search_hit peptide="{}" massdiff="{}" calc_neutral_pep_mass="{}" peptide_next_aa="{}" num_missed_cleavages="{}" num_tol_term="{}" num_tot_proteins="{}" tot_num_ions="{}" hit_rank="{}" num_matched_ions="{}" protein="{}" peptide_prev_aa="{}" is_rejected="{}" protein_descr="{}">""".format(
        data["peptide"], data["massdiff"], data["calc_neutral_pep_mass"], data["num_missed_cleavages"],
        data["num_tot_proteins"], data["tot_num_ions"], data["hit_rank"], data["num_matched_ions"],
        data["is_rejected"], data["protein"], data["protein_descr"], data["num_tol_term"],
        data["peptide_prev_aa"], data["peptide_next_aa"]
        )
        return startTag
    
    def getAlternativeTag(self, data):
        tag = """<alternative_protein protein="{}" protein_descr="{}" num_tol_term="{}" peptide_prev_aa="{}" peptide_next_aa="{}"/>""".format(
            data["protein"], data["protein_descr"], data["num_tol_term"], data["peptide_prev_aa"], data["peptide_next_aa"]
        )
        
        return tag
    
    def getModification_infoStartTag(self, data):
        key, values = list(data.items())[0]
        startTag = """<modification_info modified_peptide="{}">""".format(key)
        for v in values:
            startTag += """<mod_aminoacid_mass mass="{}" position="{}"/>""".format(v["mass"], v["position"])

        return startTag
    
    
    def getSeachScoreTags(self, data):
        tag = ""
        for k,v in data.items():
            tag += """<search_score name="{}" value="{}"/>""".format(k, v)
        return tag
    
    def getAnalysisResultStartTag(self, data):
        startTag = """<analysis_result analysis="peptideprophet"><peptideprophet_result probability="{}" all_ntt_prob="{}">""".format(
                                           data["probability"], 
                                           tuple(data["all_ntt_prob"])
                                           )
        return startTag
    
    
    def getScoreSummaryStartTag(self, data):
        startTag = "<search_score_summary>"

        for k, v in data.items():
            startTag += """<parameter name="{}" value="{}"/>""".format(k,v)

        return startTag
    
   